In [ ]:
import pandas as pd
import nltk
import string
import re
import numpy as np

In [ ]:
data_dir   = "../data/"
kaggle_dir = data_dir + "Kaggle/"
print(kaggle_dir)

In [ ]:
kaggle_true_df = pd.read_csv(kaggle_dir + "True.csv.zip", compression = "zip")
kaggle_true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
kaggle_true_df.shape

(21417, 4)

In [ ]:
kaggle_fake_df = pd.read_csv(kaggle_dir + "Fake.csv.zip", compression = "zip")
kaggle_fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
kaggle_fake_df.shape

(23481, 4)

In [ ]:
def remove_unencoded_text(text):
    """
    Removes characters that are not UTF-8 encodable.
    """
    return "".join([i if ord(i) < 128 else "" for i in text])

def is_allowed_word(word, stopwords, min_word_len):
    """
    Checks if word is allowed based on inclusion in stopwords and length.
    """
    stopwords_allowed = word not in stopwords
    length_allowed = len(word) >= min_word_len
    return stopwords_allowed and length_allowed

def preprocess(text, stopwords=set(nltk.corpus.stopwords.words("english")),
               stem=True, lemmatize=False, keep_alt_forms=False, min_word_len=1):
    '''
    Standardized preprocessing of a line of text.
    '''

    # remove non utf-8 characters
    text = remove_unencoded_text(text)

    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # convert all whitespace to spaces for splitting
    whitespace_pattern = re.compile(r"\s+")
    text = re.sub(whitespace_pattern, " ", text)

    # lowercase the input
    text = text.lower()

    # split into words
    words = text.split(" ")

    # stem and/or lemmatize words
    # filtering stopwords, numbers, and word lengths as required
    stemmer = nltk.stem.porter.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()

    if stem and lemmatize:
        words = [
            [word, stemmer.stem(word), lemmatizer.lemmatize(word)]
            for word in words if is_allowed_word(
                word, stopwords, min_word_len)]
    elif stem:
        words = [
            [word, stemmer.stem(word)]
            for word in words if is_allowed_word(
                word, stopwords, min_word_len)]
    elif lemmatize:
        words = [
            [word, lemmatizer.lemmatize(word)]
            for word in words if is_allowed_word(
                word, stopwords, min_word_len)]
    else:
        words = [
            word for word in words if is_allowed_word(
                word, stopwords, min_word_len)]

    if stem or lemmatize:
        if keep_alt_forms:
            # return both original and stemmed/lemmatized words
            # as long as stems/lemmas are unique
            words = [w for word in words for w in set(word)]
        else:
            # return only requested stems/lemmas
            # if both stemming and lemmatizing, return only lemmas
            words = list(zip(*words))[-1]

    return " ".join(words)

In [ ]:
labels = np.append(np.ones(len(kaggle_true_df)), np.zeros(len(kaggle_fake_df)), axis = 0) # 1 - true news, 0 - fake news
aggregate_df = kaggle_true_df.append(kaggle_fake_df)
preprocessed_text = []
for i in range(len(labels)):
    text_ = aggregate_df.iloc[i,:2].values
    try:
        title = preprocess(text_[0])
    except:
        continue ## No title
    try:
        body = preprocess(text_[1])
    except:
        continue ## No body
    preprocessed_text.append([title, body, labels[i]])

In [ ]:
processed_dataframe = pd.DataFrame.from_records(preprocessed_text, columns = ["Title", "Body", "Label"])

In [ ]:
processed_dataframe.to_csv(data_dir+"preprocessed_text_w_labels.csv")